In [117]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pydicom
import cv2
from math import ceil
# import json

# from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler,LabelEncoder, label_binarize
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report

# from imblearn.over_sampling import RandomOverSampler
import torch
from torch.utils.data import DataLoader,TensorDataset
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt
import seaborn as sns

# from helpers import get_ddsm_table, get_INBreast_table, get_VinDR_table, mask_image, run_intensity_functions, run_glcm_features, get_master_df, process_image
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore")

In [68]:
from helpers import get_ddsm_table, get_INBreast_table, get_VinDR_table, get_master_df, process_image, mask_image

In [43]:
ddsm_df = get_ddsm_table("../DDSM")
INbreast_df = get_INBreast_table("../INbreast Release 1.0")
vindr_df = get_VinDR_table("../VinDr")
master_df = get_master_df(vindr_df, ddsm_df, INbreast_df)

In [45]:
master_df.replace({'breast_density': {'DENSITY A': 1, 'DENSITY B': 2, 'DENSITY C': 3, 'DENSITY D': 4}}, inplace=True)
master_df['breast_density'] = pd.to_numeric(master_df['breast_density'], errors='coerce')
master_df.dropna(subset=['breast_density'], inplace=True)
master_df['breast_density'] = master_df['breast_density'] - 1

In [118]:
def data_generator(df, batch_size, process=False):
    num_samples = len(df)
    image_size = (512,512)
    while True:
        for offset in range(0, num_samples, batch_size):
            batch_df = df.iloc[offset:offset + batch_size]
            images = []
            labels = []
            
            for _, row in batch_df.iterrows():
                dicom_path = row["full_path"]
                label = row["breast_density"]
                vendor = row["Manufacturer"]
                dicom_data = pydicom.dcmread(dicom_path)
                image = dicom_data.pixel_array
                image = mask_image(image)
                
                if process:
                    image = process_image(image, vendor) # custom processing

                image = (image - np.mean(image))/np.std(image)
                image = cv2.resize(image, image_size)
                images.append(image)
                labels.append(label)
            
            # Convert to numpy arrays
            images = np.array(images).reshape(-1, 1, *image_size)  # Add channel dimension
            labels = np.array(labels)
            
            yield images, labels

In [119]:
batch_size = 32

train_df, test_df = train_test_split(master_df, test_size=0.2, random_state=42)

# Create generators
train_gen = data_generator(train_df, batch_size)
test_gen = data_generator(test_df, batch_size)

# Get the number of steps per epoch
train_steps = ceil(len(train_df) // batch_size)
test_steps = ceil(len(test_df) // batch_size)


In [120]:
# Get a batch from the generator
images, labels = next(train_gen)

# Print the shapes of the images and labels
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

Images shape: (32, 1, 512, 512)
Labels shape: (32,)


In [100]:
len(val_df)

516

In [92]:
len(master_df)

2884

In [86]:
X = master_df.drop(columns=['breast_density'])
y = master_df['breast_density']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [87]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [89]:
len(X_train)

2307

In [ ]:
# build data loaders
    # train test split
    # X and Y
# find shape of X by vendor so we can decide on input dimensions
# build model
# train model
# test model